In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%reload_ext autoreload
%autoreload 2

### Loading data

In [44]:
from datasets import get_dataset_split_names,load_dataset

path="Bena345/cdc-diabetes-health-indicators"

split_names=get_dataset_split_names(path)
train_size=10000
test_size=5000

diabetes=load_dataset(path,split=[f'{split_names[0]}[:{train_size}]', f'{split_names[1]}[:{train_size}]'])
diabetes


[Dataset({
     features: ['ID', 'BMI', 'PhysHlth', 'Age', 'HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth', 'MentHlth', 'DiffWalk', 'Sex', 'Education', 'Income', 'Diabetes_binary'],
     num_rows: 10000
 }),
 Dataset({
     features: ['ID', 'BMI', 'PhysHlth', 'Age', 'HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth', 'MentHlth', 'DiffWalk', 'Sex', 'Education', 'Income', 'Diabetes_binary'],
     num_rows: 10000
 })]

### splitting data

In [45]:
diabetes_train_dataframe=diabetes[0].to_pandas()
diabetes_test_dataframe=diabetes[1].to_pandas()

In [46]:
diabetes_train_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    10000 non-null  int64  
 1   BMI                   10000 non-null  float32
 2   PhysHlth              10000 non-null  float32
 3   Age                   10000 non-null  object 
 4   HighBP                10000 non-null  object 
 5   HighChol              10000 non-null  object 
 6   CholCheck             10000 non-null  object 
 7   Smoker                10000 non-null  object 
 8   Stroke                10000 non-null  object 
 9   HeartDiseaseorAttack  10000 non-null  object 
 10  PhysActivity          10000 non-null  object 
 11  Fruits                10000 non-null  object 
 12  Veggies               10000 non-null  object 
 13  HvyAlcoholConsump     10000 non-null  object 
 14  AnyHealthcare         10000 non-null  object 
 15  NoDocbcCost         

In [47]:
diabetes_train_dataframe.head(5)

,ID,BMI,PhysHlth,Age,HighBP,HighChol,CholCheck,Smoker,Stroke,HeartDiseaseorAttack,...,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,DiffWalk,Sex,Education,Income,Diabetes_binary
0,114414,29.0,0.0,65 to 69,0,1,1,0,0,0,...,0,1,1,Poor,0,0,0,6,7,Non-Diabetic
1,168896,32.0,0.0,80 or older,1,1,1,0,0,0,...,0,1,0,Poor,0,0,0,Some College Degree,4,Non-Diabetic
2,68354,25.0,5.0,65 to 69,1,0,1,1,0,0,...,0,1,0,Fair,0,0,1,6,2,Non-Diabetic
3,121194,24.0,0.0,80 or older,1,0,1,0,0,0,...,0,1,0,Very Good,0,1,0,Advanced Degree,5,Non-Diabetic
4,141150,31.0,0.0,25 to 29,0,0,1,0,0,1,...,0,1,0,Very Good,5,0,0,6,6,Diabetic
